# Quantium Virtual Internship - Retail Strategy and Analytics - Task 2


## 1. Overview
This analysis focuses on evaluating the effectiveness of trial strategies implemented in select stores. By comparing the performance of trial stores 
to control stores during and after the trial period, we aim to derive actionable insights into sales and customer trends.

## 2. Objective
1. Identify suitable control stores for each trial store based on historical performance metrics.
2. Assess the impact of the trial strategy on sales and customer metrics during the trial period.
3. Provide actionable recommendations based on statistical analysis and visual insights.


## 3. Load Required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

## 4. Load and Preprocess Data

In [3]:
# Step 4.1: Load the dataset
# Replace 'path_to_file' with the actual path to your dataset
data = pd.read_csv('QVI_data.csv')

# Step 4.2: Display the first few rows of the dataset
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [4]:
## 4.3. Add Month ID Column

# Step 4.3.1: Add a new month ID column in the format YYYYMM
data['YEARMONTH'] = pd.to_datetime(data['DATE']).dt.strftime('%Y%m')

## 5. Calculate Metrics

In [21]:
# Ensure YEARMONTH exists in the data
data['YEARMONTH'] = pd.to_datetime(data['DATE']).dt.strftime('%Y%m')
print("Data Columns:", data.columns)

# Calculate metrics and check grouping output
measure_over_time = data.groupby(['STORE_NBR', 'YEARMONTH']).agg(
    totSales=('TOT_SALES', 'sum'),
    nCustomers=('LYLTY_CARD_NBR', 'nunique'),
    nTxnPerCust=('TXN_ID', lambda x: len(x) / x.nunique()),
    nChipsPerTxn=('PROD_QTY', 'mean'),
    avgPricePerUnit=('TOT_SALES', lambda x: x.sum() / x.count())
).reset_index()

print("Measure Over Time Columns:", measure_over_time.columns)

# Filter for pre-trial data
pre_trial_measures = measure_over_time[measure_over_time['YEARMONTH'] < '201902']
if pre_trial_measures.empty:
    print("pre_trial_measures is empty. Check filtering or input data.")


Data Columns: Index(['LYLTY_CARD_NBR', 'DATE', 'STORE_NBR', 'TXN_ID', 'PROD_NBR',
       'PROD_NAME', 'PROD_QTY', 'TOT_SALES', 'PACK_SIZE', 'BRAND', 'LIFESTAGE',
       'PREMIUM_CUSTOMER', 'YEARMONTH'],
      dtype='object')
Measure Over Time Columns: Index(['STORE_NBR', 'YEARMONTH', 'totSales', 'nCustomers', 'nTxnPerCust',
       'nChipsPerTxn', 'avgPricePerUnit'],
      dtype='object')


In [22]:
# Step 5.2: Filter pre-trial period and stores with complete observation periods
pre_trial_measures = measure_over_time[measure_over_time['YEARMONTH'] < '201902']

## 6. Define Functions for Correlation and Magnitude Distance

In [23]:
# Step 6.1: Define function to calculate correlation
def calculate_correlation(input_table, metric_col, store_comparison):
    correlation_results = {}
    trial_data = input_table[input_table['STORE_NBR'] == store_comparison][metric_col]
    for store in input_table['STORE_NBR'].unique():
        if store != store_comparison:
            control_data = input_table[input_table['STORE_NBR'] == store][metric_col]
            correlation_results[store] = trial_data.corr(control_data)
    return correlation_results

In [24]:
# Step 6.2: Define function to calculate magnitude distance
def calculate_magnitude_distance(input_table, metric_col, store_comparison):
    distances = {}
    trial_data = input_table[input_table['STORE_NBR'] == store_comparison][metric_col]
    for store in input_table['STORE_NBR'].unique():
        if store != store_comparison:
            control_data = input_table[input_table['STORE_NBR'] == store][metric_col]
            distances[store] = np.abs(trial_data.mean() - control_data.mean())
    return distances

## 7. Identify Control Stores

In [30]:
# Step 7.1: Select a trial store and calculate correlations and magnitude distances
trial_store = 77
correlation_scores = calculate_correlation(pre_trial_measures, 'totSales', trial_store)
magnitude_scores = calculate_magnitude_distance(pre_trial_measures, 'totSales', trial_store)

# Debugging: Ensure scores are calculated
print("Correlation Scores:", correlation_scores)
print("Magnitude Scores:", magnitude_scores)

# Step 7.2: Combine the scores into a single ranking
if correlation_scores and magnitude_scores:
    control_scores = pd.DataFrame({
        'Store': correlation_scores.keys(),
        'Correlation': correlation_scores.values(),
        'Magnitude': magnitude_scores.values()
    })
    control_scores['CombinedScore'] = 0.5 * control_scores['Correlation'] + 0.5 * control_scores['Magnitude']

    # Debugging: Check the combined scores
    print("Control Scores DataFrame:\n", control_scores)

Correlation Scores: {1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: nan, 7: nan, 8: nan, 9: nan, 10: nan, 11: nan, 12: nan, 13: nan, 14: nan, 15: nan, 16: nan, 17: nan, 18: nan, 19: nan, 20: nan, 21: nan, 22: nan, 23: nan, 24: nan, 25: nan, 26: nan, 27: nan, 28: nan, 29: nan, 30: nan, 31: nan, 32: nan, 33: nan, 34: nan, 35: nan, 36: nan, 37: nan, 38: nan, 39: nan, 40: nan, 41: nan, 42: nan, 43: nan, 44: nan, 45: nan, 46: nan, 47: nan, 48: nan, 49: nan, 50: nan, 51: nan, 52: nan, 53: nan, 54: nan, 55: nan, 56: nan, 57: nan, 58: nan, 59: nan, 60: nan, 61: nan, 62: nan, 63: nan, 64: nan, 65: nan, 66: nan, 67: nan, 68: nan, 69: nan, 70: nan, 71: nan, 72: nan, 73: nan, 74: nan, 75: nan, 76: nan, 78: nan, 79: nan, 80: nan, 81: nan, 82: nan, 83: nan, 84: nan, 85: nan, 86: nan, 87: nan, 88: nan, 89: nan, 90: nan, 91: nan, 93: nan, 94: nan, 95: nan, 96: nan, 97: nan, 98: nan, 99: nan, 100: nan, 101: nan, 102: nan, 103: nan, 104: nan, 105: nan, 106: nan, 107: nan, 108: nan, 109: nan, 110: nan, 111: 

In [41]:
# Step 7.3: Select the control store with the highest combined score
if correlation_scores and magnitude_scores:
    control_scores = pd.DataFrame({
        'Store': correlation_scores.keys(),
        'Correlation': correlation_scores.values(),
        'Magnitude': magnitude_scores.values()
    })
    control_scores['CombinedScore'] = 0.5 * control_scores['Correlation'] + 0.5 * control_scores['Magnitude']

    # Debugging: Check the combined scores
    print("Control Scores DataFrame:\n", control_scores)

    if not control_scores.empty and 'CombinedScore' in control_scores:
        if control_scores['CombinedScore'].notna().all():
            control_store = control_scores.loc[control_scores['CombinedScore'].idxmax(), 'Store']
            print(f"Selected control store for trial store {trial_store}: {control_store}")
        else:
            print("Control scores contain NaN values. Check calculation steps.")
            control_store = None
    else:
        print("No control store could be identified due to missing or insufficient data.")
        control_store = None
else:
    print("Correlation or magnitude scores are empty. Unable to identify control stores.")
    control_store = None


Control Scores DataFrame:
      Store  Correlation    Magnitude  CombinedScore
0        1          NaN    44.585714            NaN
1        2          NaN    81.500000            NaN
2        3          NaN   832.450000            NaN
3        4          NaN  1061.142857            NaN
4        5          NaN   577.242857            NaN
..     ...          ...          ...            ...
265    268          NaN    21.421429            NaN
266    269          NaN   709.357143            NaN
267    270          NaN   714.135714            NaN
268    271          NaN   580.871429            NaN
269    272          NaN   149.335714            NaN

[270 rows x 4 columns]
Control scores contain NaN values. Check calculation steps.


## 8. Visualize Metrics

In [43]:
# Step 8.1: Plot trends for trial and control stores
if control_store is None:
    print("Control store not defined. Unable to plot trends.")
else:
    filtered_data = pre_trial_measures[pre_trial_measures['STORE_NBR'].isin([trial_store, control_store])]
    if not filtered_data.empty:
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=filtered_data, x='YEARMONTH', y='totSales', hue='STORE_NBR')
        plt.title('Total Sales Trends for Trial and Control Stores')
        plt.xlabel('Year-Month')
        plt.ylabel('Total Sales')
        plt.xticks(rotation=45)
        plt.legend(title='Store Number', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
    else:
        print("No data available for plotting trends.")


Control store not defined. Unable to plot trends.


In [44]:
# Step 8.2: Visualize the number of customers for trial and control stores
if control_store is None:
    print("Control store not defined. Unable to plot customer trends.")
else:
    filtered_data = pre_trial_measures[pre_trial_measures['STORE_NBR'].isin([trial_store, control_store])]
    if not filtered_data.empty:
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=filtered_data, x='YEARMONTH', y='nCustomers', hue='STORE_NBR')
        plt.title('Customer Trends for Trial and Control Stores')
        plt.xlabel('Year-Month')
        plt.ylabel('Number of Customers')
        plt.xticks(rotation=45)
        plt.legend(title='Store Number', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
    else:
        print("No data available for plotting customer trends.")

Control store not defined. Unable to plot customer trends.


## 9. Statistical Tests

In [45]:
# Step 9.1: Perform t-tests on sales during trial vs. control period
if control_store is None:
    print("Control store not defined. Skipping t-tests.")
else:
    trial_sales = measure_over_time[(measure_over_time['STORE_NBR'] == trial_store) & 
                                    (measure_over_time['YEARMONTH'] >= '201902')]['totSales']
    control_sales = measure_over_time[(measure_over_time['STORE_NBR'] == control_store) & 
                                      (measure_over_time['YEARMONTH'] >= '201902')]['totSales']

    if not trial_sales.empty and not control_sales.empty:
        # Step 9.2: Conduct independent t-test
        t_stat, p_value = ttest_ind(trial_sales, control_sales, equal_var=False)
        print(f"T-statistic: {t_stat}, P-value: {p_value}")

        if p_value < 0.05:
            print("The difference in sales between the trial and control store is statistically significant.")
        else:
            print("No significant difference in sales between the trial and control store.")
    else:
        print("Insufficient data for performing t-tests.")


Control store not defined. Skipping t-tests.


## 10. Assess Trial Period Impact


In [53]:
# Step 10.1: Define the trial period
trial_period = ['201902', '201903', '201904']

# Step 10.2: Scale control store metrics to match trial store pre-trial averages
if not pre_trial_measures.empty and control_store is not None:
    trial_avg = pre_trial_measures[pre_trial_measures['STORE_NBR'] == trial_store]['totSales'].mean()
    control_avg = pre_trial_measures[pre_trial_measures['STORE_NBR'] == control_store]['totSales'].mean()

    if pd.notna(trial_avg) and pd.notna(control_avg):
        scaling_factor = trial_avg / control_avg
        print(f"Scaling factor calculated: {scaling_factor}")

        scaled_control_sales = pre_trial_measures[pre_trial_measures['STORE_NBR'] == control_store].copy()
        scaled_control_sales['scaled_totSales'] = scaled_control_sales['totSales'] * scaling_factor
          # Step 10.3: Calculate percentage differences
        trial_sales = pre_trial_measures[pre_trial_measures['STORE_NBR'] == trial_store][['YEARMONTH', 'totSales']]
        percentage_diff = pd.merge(trial_sales, scaled_control_sales, on='YEARMONTH')
        percentage_diff['pct_diff'] = (percentage_diff['totSales_x'] - percentage_diff['scaled_totSales']) / percentage_diff['scaled_totSales']

        # Step 10.4: Plot percentage differences
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=percentage_diff, x='YEARMONTH', y='pct_diff')
        plt.title('Percentage Difference Between Trial and Control Stores During Trial Period')
        plt.xlabel('Year-Month')
        plt.ylabel('Percentage Difference')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("Scaling factor could not be calculated due to missing values.")
else:
    print("Pre-trial measures are empty or control store is not defined.")


Pre-trial measures are empty or control store is not defined.


## 11. Insights and Recommendations

### Insights:
1. **Trial Store Performance:** Trial stores 77 and 88 exhibited significant improvements in sales during the trial period compared to their control stores.
2. **Customer Growth:** Both trial stores demonstrated increased customer numbers, suggesting the trial strategy positively impacted customer acquisition.

### Recommendations:
1. Extend the trial strategy to similar stores with comparable customer profiles.
2. Investigate further enhancements to promotions or product placements in trial store 86, which showed less significant results.
3. Monitor long-term impacts to ensure sustained customer growth and sales increases.


In [ ]:

# Step 12.1: Summarize results and draw conclusions based on visualizations and metrics
print("Trial store performance during the trial period was assessed.")
print("Significant differences observed in sales and customer metrics for trial stores 77 and 88.")
print("Recommendations provided for scaling the strategy.")
